In [ ]:
!pip install pydub

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from os import *
import librosa
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Flatten
import tensorflow.keras.backend as K
from tensorflow.keras.layers.experimental import preprocessing
from keras.utils import to_categorical

import os
import matplotlib
matplotlib.use('Agg') # No pictures displayed 
import pylab
import librosa
import librosa.display
import numpy as np
from pydub import AudioSegment

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
pip install urllib.request

ERROR: Could not find a version that satisfies the requirement urllib.request (from versions: none)
ERROR: No matching distribution found for urllib.request


In [ ]:

import os
import urllib.request

import tarfile
from bs4 import BeautifulSoup
import lxml.html as html

In [ ]:
#Сылка на  открытую базу данных c речевыми аудио и транскрипциями , выбираем русский язык и частоту 16 кГц

voxforge_url = 'http://www.repository.voxforge1.org/downloads/SpeechCorpus/Trunk/Audio/Main/16kHz_16bit/'
target_folder = '/content/gdrive/My Drive/voxforge_eng'

In [ ]:

def download_file(url, target_folder):
    """
    Downloads and extracts a tar file given a URL and a target folder.
    """
    stream = urllib.request.urlopen(url)
    tar = tarfile.open(fileobj=stream, mode="r|gz")

    for item in tar:
        tar.extract(item, target_folder)

In [ ]:
page= html.parse(voxforge_url).getroot()

In [ ]:
e=html.tostring(page)

In [ ]:
e

b'<html><head><title>VoxForge Repository</title>\n\n\t<style type="text/css">\n\t.siteFunctions {\n\t\ttext-align: right;\n\t}\n\t.copyright {\n\t\ttest-align: left;\n\t\tcolor: #2E3436;\n\t\tfont-family: sans-serif;\n                font-size: small;\n\t}\n\n\tbody {\n\t\tfont-family: "DejaVu Sans", "Lucida Sans Unicode", sans-serif;\n\t\tfont-weight:\tnormal;\n\t\tword-spacing:\tnormal;\n\t\tletter-spacing:\tnormal; \n\t\ttext-transform:\tnone;\n\t\tfont-size: medium;\n                text-align: justify;\n\t}\n\th2 {\n\t\tfont-size:\t1.5em;\n\t\tfont-weight:\t700;\n\t\tmargin-top:1em;\n\t\tmargin-bottom:0.8em;\n\t}\n\th3 {\n\t\tfont-size:\t1.1em;\n\t\tfont-weight:\t600;\n\t\tmargin-top:1em;\n\t\tmargin-bottom:0.4em;\n\t}\n\tp, ol, ul {\n\t\tfont-size:\t1em;\n\t\tmargin-top:0.4em;\n\t\tmargin-bottom:0.4em;\n\t}\t\n\t.heading {\n\t\tbackground-color: #555753;\n                color: #D3D7CF;\n\t\tfont-size: 40px;\n\t\tvertical-align: bottom;\n\t}\n\t.logo {\n\t\twidth: 100px; \n\t\tfl

In [ ]:
soup = BeautifulSoup(e)

In [ ]:
soup

<html><head><title>VoxForge Repository</title>
<style type="text/css">
	.siteFunctions {
		text-align: right;
	}
	.copyright {
		test-align: left;
		color: #2E3436;
		font-family: sans-serif;
                font-size: small;
	}

	body {
		font-family: "DejaVu Sans", "Lucida Sans Unicode", sans-serif;
		font-weight:	normal;
		word-spacing:	normal;
		letter-spacing:	normal; 
		text-transform:	none;
		font-size: medium;
                text-align: justify;
	}
	h2 {
		font-size:	1.5em;
		font-weight:	700;
		margin-top:1em;
		margin-bottom:0.8em;
	}
	h3 {
		font-size:	1.1em;
		font-weight:	600;
		margin-top:1em;
		margin-bottom:0.4em;
	}
	p, ol, ul {
		font-size:	1em;
		margin-top:0.4em;
		margin-bottom:0.4em;
	}	
	.heading {
		background-color: #555753;
                color: #D3D7CF;
		font-size: 40px;
		vertical-align: bottom;
	}
	.logo {
		width: 100px; 
		float: left;
		text-align: left;
	}
	.logo img {
		border: 0px;
	}
	img {
		border: 0px;
	}
	.clickableicons {
	}
	.endFloat {
		cl

In [ ]:
# список всех ссылок
links = soup.findAll('a')

In [ ]:
links

[<a href="http://www.voxforge.org"><img alt="VoxForge Repository" src="http://www.voxforge.org/uploads/8k/N8/8kN884Cd96cmBZxRlzmbzQ/voxforge-logo.jpg"/> </a>,
 <a class="horizontalMenu" href="http://www.voxforge.org/home">Home</a>,
 <a class="horizontalMenu" href="http://www.voxforge.org/home/read">Read</a>,
 <a class="horizontalMenu" href="http://www.voxforge.org/home/listen">Listen</a>,
 <a class="horizontalMenu" href="http://www.voxforge.org/home/forums">Forums</a>,
 <a class="horizontalMenu" href="http://www.voxforge.org/home/dev">Dev</a>,
 <a class="horizontalMenu" href="http://www.voxforge.org/home/downloads">Downloads</a>,
 <a class="horizontalMenu" href="http://www.voxforge.org/home/about">About</a>,
 <a href="?C=N;O=D">Name</a>,
 <a href="?C=M;O=A">Last modified</a>,
 <a href="?C=S;O=A">Size</a>,
 <a href="/downloads/SpeechCorpus/Trunk/Audio/Main/">Parent Directory</a>,
 <a href="1028-20100710-hne.tgz">1028-20100710-hne.tgz</a>,
 <a href="1337ad-20170321-ajg.tgz">1337ad-201703

In [ ]:
speaker_refs=[]
for l in links:
    if '.tgz'in str(l):
        speaker_refs.append(l)

In [ ]:
speaker_refs

[<a href="1028-20100710-hne.tgz">1028-20100710-hne.tgz</a>,
 <a href="1337ad-20170321-ajg.tgz">1337ad-20170321-ajg.tgz</a>,
 <a href="1337ad-20170321-tkg.tgz">1337ad-20170321-tkg.tgz</a>,
 <a href="1337ad-20170321-ynk.tgz">1337ad-20170321-ynk.tgz</a>,
 <a href="1snoke-20120412-hge.tgz">1snoke-20120412-hge.tgz</a>,
 <a href="23yipikaye-20100807-ujm.tgz">23yipikaye-20100807-ujm.tgz</a>,
 <a href="2old2play-20110606-hcn.tgz">2old2play-20110606-hcn.tgz</a>,
 <a href="2old2play-20110606-wlt.tgz">2old2play-20110606-wlt.tgz</a>,
 <a href="314piwm-20130617-xuo.tgz">314piwm-20130617-xuo.tgz</a>,
 <a href="AJN-20131002-zcl.tgz">AJN-20131002-zcl.tgz</a>,
 <a href="AT-20130718-lws.tgz">AT-20130718-lws.tgz</a>,
 <a href="Aaron-20080318-kdl.tgz">Aaron-20080318-kdl.tgz</a>,
 <a href="Aaron-20080318-lbb.tgz">Aaron-20080318-lbb.tgz</a>,
 <a href="Aaron-20080318-lbk.tgz">Aaron-20080318-lbk.tgz</a>,
 <a href="Aaron-20080318-liy.tgz">Aaron-20080318-liy.tgz</a>,
 <a href="Aaron-20080318-ngh.tgz">Aaron-2008

In [ ]:
# Скачиваем файлы на диск
for i, ref in enumerate(speaker_refs):
    print('Downloading {} / {} files'.format(i, len(speaker_refs)))
#     download_file(voxforge_url + '/' + ref, target_folder)
    sss=str(ref)
    ssss1=sss[sss.find('"'):sss.rfind('"')]
    dd=voxforge_url  + str(ssss1)
    print(dd)
    download_file(dd, target_folder)

http://www.repository.voxforge1.org/downloads/SpeechCorpus/Trunk/Audio/Main/16kHz_16bit/"1028-20100710-hne.tgz
http://www.repository.voxforge1.org/downloads/SpeechCorpus/Trunk/Audio/Main/16kHz_16bit/"1337ad-20170321-ajg.tgz
http://www.repository.voxforge1.org/downloads/SpeechCorpus/Trunk/Audio/Main/16kHz_16bit/"1337ad-20170321-tkg.tgz
http://www.repository.voxforge1.org/downloads/SpeechCorpus/Trunk/Audio/Main/16kHz_16bit/"1337ad-20170321-ynk.tgz
http://www.repository.voxforge1.org/downloads/SpeechCorpus/Trunk/Audio/Main/16kHz_16bit/"1snoke-20120412-hge.tgz
http://www.repository.voxforge1.org/downloads/SpeechCorpus/Trunk/Audio/Main/16kHz_16bit/"23yipikaye-20100807-ujm.tgz
http://www.repository.voxforge1.org/downloads/SpeechCorpus/Trunk/Audio/Main/16kHz_16bit/"2old2play-20110606-hcn.tgz
http://www.repository.voxforge1.org/downloads/SpeechCorpus/Trunk/Audio/Main/16kHz_16bit/"2old2play-20110606-wlt.tgz
http://www.repository.voxforge1.org/downloads/SpeechCorpus/Trunk/Audio/Main/16kHz_16bit/

HTTPError: ignored

In [ ]:
eng_wav = '/content/gdrive/My Drive/voxforge_eng'
eng_folderslist = sorted(listdir(eng_wav))

In [ ]:
eng_folderslist

In [ ]:
pic_eng = '/content/gdrive/My Drive/VOX_PIC_ENG'

In [ ]:
i = 0
for folder in eng_folderslist:
  a = str(folder)
  one_folder = eng_wav + '/' + folder + '/' + 'wav'
  wavlist = listdir(one_folder)
  for files in wavlist:
    b = str(files)
    i = i + 1
    print(b)
    sig, fs = librosa.load(one_folder + '/' + b )   
   
# make pictures name 
    b = b.replace('.wav', str(i) + '.jpg')
    save_path = pic_eng+'/'+ b
    
    pylab.axis('off') # no axis
    pylab.axes([0., 0., 1., 1.], frameon=False, xticks=[], yticks=[]) # Remove the white edge
    S = librosa.feature.melspectrogram(y=sig, sr=fs)
    librosa.display.specshow(librosa.power_to_db(S, ref=np.max))
    pylab.savefig(save_path, bbox_inches=None, pad_inches=0)
    pylab.close()


Выходные данные были обрезаны до нескольких последних строк (5000).
b0166.wav
b0160.wav
b0157.wav
b0165.wav
b0162.wav
b0158.wav
b0163.wav
b0164.wav
a0151.wav
a0156.wav
a0154.wav
a0152.wav
a0155.wav
a0148.wav
a0149.wav
a0153.wav
a0147.wav
a0150.wav
a0290.wav
a0286.wav
a0285.wav
a0283.wav
a0284.wav
a0289.wav
a0288.wav
a0287.wav
a0281.wav
a0282.wav
a0403.wav
a0404.wav
a0396.wav
a0405.wav
a0399.wav
a0397.wav
a0402.wav
a0401.wav
a0400.wav
a0398.wav
a0580.wav
a0578.wav
a0577.wav
a0576.wav
a0574.wav
a0581.wav
a0573.wav
a0579.wav
a0575.wav
ar-11.wav
ar-15.wav
ar-09.wav
ar-06.wav
ar-07.wav
ar-13.wav
ar-14.wav
ar-10.wav
ar-12.wav
ar-08.wav
b0486.wav
b0485.wav
b0488.wav
b0482.wav
b0481.wav
b0483.wav
b0487.wav
b0484.wav
b0479.wav
b0480.wav
a0550.wav
a0551.wav
a0548.wav
a0555.wav
a0547.wav
a0549.wav
a0553.wav
a0552.wav
a0554.wav
a0546.wav
b0190.wav
b0186.wav
b0193.wav
b0191.wav
b0187.wav
b0189.wav
b0192.wav
b0195.wav
b0188.wav
b0194.wav
b0239.wav
b0237.wav
b0236.wav
b0233.wav
b0238.wav
b0241.wav
b0